In [3]:
#Welcome my dear colleague :) Change the name of the sheet and measurement number
# Press CTRL+F9
# Wait and when a prompt from google drive appears, press "Authenticate"

# Measurement number 1-6
cislo_merania = 4

# Sheet name
sheet_name = 'TSEI automatika JUN 2 pacientTLAC'

In [4]:
!pip install fillpdf
# https://pypi.org/project/fillpdf/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 60.3 MB/s eta 0:00:00


In [5]:
#from google.colab import drive
#drive.mount('/mntDrive/')

In [6]:
import pandas as pd

In [7]:
from google.colab import auth
auth.authenticate_user()



In [8]:
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [9]:
! gdown --id 1eWhRtHsjQUBqvMDdDFJt80CEKf_rPnhN

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1eWhRtHsjQUBqvMDdDFJt80CEKf_rPnhN
To: /content/template.pdf
100% 452k/452k [00:00<00:00, 99.7MB/s]


In [10]:
worksheet = gc.open(sheet_name)

worksheet_list = worksheet.worksheets()
worksheet_list

[<Worksheet 'Spolu Vyhodnotenie' id:1283777913>,
 <Worksheet '1f1_8p' id:1478099426>,
 <Worksheet '1f2_8p' id:1568330821>,
 <Worksheet '2f1_4p_priame' id:1613645308>,
 <Worksheet '2f2_4P_sikme' id:1759751675>,
 <Worksheet '3f1_4p_priame' id:1940248079>,
 <Worksheet '3f2_4P_sikme' id:1605614275>,
 <Worksheet 'Kumulativna Dávka' id:1043066584>,
 <Worksheet 'Automatické Vyplnenie Návod' id:240575350>,
 <Worksheet 'Kalibrácia' id:1465413247>]

In [11]:
worksheet_sheet = worksheet.get_worksheet(cislo_merania)

In [12]:
worksheet_sheet

<Worksheet '2f2_4P_sikme' id:1759751675>

In [13]:
#get_all_values gives a list of rows
rows = worksheet_sheet.get_all_values()
#Convert to a DataFrame
df = pd.DataFrame(rows)

#set column names equal to values in row index position 0
df.columns = df.iloc[0]
#remove first row from DataFrame
df = df[1:]

df.head()

,Oblasť,TLD,Ri,,kki,Dávka,,Kq= 0.84,,
1,vrch.hlavy,61,30174,,4.3E-05,1.29,,,Dátum,30.6.2023
2,vrch.hlavy dx,62,28600,,4.4E-05,1.27,,,,
3,vrch. hlavy sin,63,4205,,4.2E-05,0.18,,,Nepoužité,"67,119, 120"
4,nad ucho dx,64,8250,,4.7E-05,0.39,,,,
5,nad ucho sin,65,24683,,4.3E-05,1.06,,MU:200 na pole,,


In [14]:
import fillpdf
from fillpdf import fillpdfs

In [15]:
input_pdf_path = "template.pdf"
output_pdf_path = f"template {cislo_merania}.pdf"

In [16]:
dict_boxes = fillpdfs.get_form_fields(input_pdf_path, sort=False, page_number=None)

In [17]:
df["Dávka"].values

array(['1.29', '1.27', '0.18', '0.39', '1.06', '1.11', '1.25', '1.26',
       '1.29', '1.19', '1.63', '0.92', '0.04', '0.73', '1.20', '1.09',
       '1.47', '1.34', '1.28', '1.15', '1.50', '0.03', '1.36', '0.86',
       '1.29', '1.34', '1.40', '1.45', '1.17', '0.59', '0.94', '1.69',
       '0.26', '1.55', '1.43', '1.75', '0.37', '0.00', '0.97', '2.05',
       '0.41', '1.48', '0.03', '1.03', '1.54', '0.98', '0.90', '1.45',
       '1.43', '1.29', '1.05', '1.44', '1.34', '1.38', '1.20', '0.06',
       '0.10', '0.00', ''], dtype=object)

In [18]:
# Fills array with Dose (davka) and count numbers (Ri)
davka = df["Dávka"].values
Ri = df["Ri"].values

In [19]:
# This walks step by ste by each fillable box
# This part of the code is painfull to see so please clean your eyes afterwards

for id, box_name in enumerate(dict_boxes):
  if (id < 57):
    dict_boxes[box_name] = davka[id]
  elif ((id>= 57) & (id<114)):
    dict_boxes[box_name] = Ri[id-57]
  else:
    dict_boxes[box_name] = davka[id-114]

In [20]:
# Fills Pdf
fillpdfs.write_fillable_pdf(input_pdf_path, output_pdf_path, dict_boxes , flatten=False)

In [21]:
# Dowload PDF

from google.colab import files
files.download(output_pdf_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>